# Chains

On a vu qu'il était possible d'utiliser une chaine simple (LLMChain) pour faire de la génération de texte. Mais on peut aussi utiliser des chaines plus complexes pour réaliser des tâches plus complexes.

La librairie langchain propose plusieurs types de chaines.

In [2]:
# base packages
import os
import pandas as pd

# load the environment variables
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# import model
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
    openai_api_version="2023-09-01-preview",
    azure_endpoint=os.getenv('AZURE_API_ENDPOINT'),
    api_key=os.getenv('AZURE_OPENAI_KEY'),
    azure_deployment=os.getenv('OPENAI_DEPLOYMENT_NAME'),
    model_name=os.getenv('OPENAI_MODEL_NAME'),
    model_version=os.getenv('OPENAI_API_VERSION')
)

C:\Users\sebas\AppData\Local\Temp\ipykernel_67168\3584669270.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## SimpleSequentialChain



In [10]:
# read data
df = pd.read_csv('../data/Data.csv')
df.head(6)

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \r\n,Je trouve le goût médiocre. La mousse ne tient...


In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain, SimpleSequentialChain

# defining prompt templates
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)

# defining simple chains
# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

# chaining above chains
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True # to get what is going on
                                            )
# input definition
product = df['Product'].iloc[0]

# running the chain
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
Royal Linens.
Royal Linens is a high-quality bedding company that offers luxurious and comfortable linens that are fit for royalty.

> Finished chain.


{'input': 'Queen Size Sheet Set',
 'output': 'Royal Linens is a high-quality bedding company that offers luxurious and comfortable linens that are fit for royalty.'}

## SequentialChain

On souhaite réaliser une chaine qui prend en entrée la revue d'un produit en anglais et qui retourne une réponse dans la même langue que la revue.

![Alt text](image-4.png)

In [8]:
## prompt templates
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)

#p prompt template 2 : summerization
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}" # the output_key previously defined (in the first chain)
)

# prompt template 3: language identification
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}" # the original review
)

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
    "\n\nOnly provide the message without teh summary."
)


## chains definitions
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, 
                     prompt=first_prompt, 
                     output_key="English_Review" # here we specify the output key (OPTIONAL: Helps to keep track of the output)
                    )


# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, 
                     prompt=second_prompt, 
                     output_key="summary"
                    )


# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, 
                       prompt=third_prompt,
                       output_key="language"
                      )


# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, 
                      prompt=fourth_prompt,
                      output_key="followup_message"
                     )

# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)
review = df.Review[5]

# running the chain
overall_chain.invoke(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit?!",
 'summary': 'The reviewer found the taste of the product mediocre and the foam strange, and suspects that it may be an old batch or counterfeit.',
 'followup_message': "Je suis désolé d'apprendre que vous n'avez pas apprécié le goût du produit et que la mousse était étrange. Nous prenons très au sérieux la qualité de nos produits et nous vérifierons si ce lot est vieux ou s'il s'agit d'une contrefaçon. Veuillez nous contacter pour que nous puissions vous aider à résoudre ce problème."}